## Create table 4

This code reads the output files from the models and compiles table 4.

These are the models that compute the unilateral trade liberalization. The model names in the paper map into the code according to: 

   * Financial autarky: \Unilateral_Bench_Autarky
   * Non-contingent bond: \Unilateral_Bench_Bond
   * Complete markets: \Unilateral_Bench_Complete
   * The no-cost version of the benchmark model with a non-contingent bond: \Unilateral_NoCost_Bond
   
   
The following functions extract the numbers needed in the tables from the model output. 

In [1]:
import numpy as np
import pandas as pd

### Read in the data

These two functions extract the data we need from the model.out files.

In [2]:
def get_welfare(file_path):
    fr = open(file_path)
    buffer = fr.readlines()
    buffer = [b.strip() for b in  buffer]
    buffer = [b for b in buffer if len(b)>0]
    
    data = {}
    for i, elm in enumerate(buffer):
        
        if elm.startswith('Dynamic gains'):
            data['welfareH'] = elm.split()[3]
            data['welfareF'] = elm.split()[4]
    return data

In [3]:
def read_transition(file_path):
    fp =  open(file_path)
    buffer = fp.readlines()
    buffer = [b.strip() for b in  buffer]
    buffer = [b for b in buffer if len(b)>0]

    trans = []
    saveit = 0
    for i, elm in enumerate(buffer):
        if elm.startswith('Year'):
            saveit = 1
            linelen = len(elm.split())
         
        # There may be crud after the transition
        if (saveit == 1) and (len(elm.split())==linelen):
            trans.append(elm.split())

    trans = pd.DataFrame(trans[1:], columns=trans[0], dtype=float)
    trans = trans.set_index('Year')
    return trans

### Extract the transitions and welfare from model output

In [4]:
filepaths = ['../Model code/Unilateral_Bench_Bond/Uni_Bench_Bond01.out',
             '../Model code/Unilateral_Bench_Autarky/Uni_Bench_Autarky01.out',
             '../Model code/Unilateral_Bench_Complete/Uni_Bench_Complete01.out',
             '../Model code/Unilateral_NoCost_Bond/Uni_NoCost_Bond01.out'
             ]
models = ['bond', 'aut', 'complete', 'nocostbond']
welfs = {}
trans = {}

for f, m in zip(filepaths, models):
    welfs[m] = get_welfare(f)
    trans[m] = read_transition(f)

### Compute steady-state to steady-state changes

You will receive a warning about taking the log of zero. It can be ignored. 

In [5]:
def compute_ss_changes(trans):  
    ss_change = np.log(trans.loc[300]/trans.loc[0])*100
    return ss_change

In [6]:
ss_changes = {}

for m in models:
    ss_changes[m] = compute_ss_changes(trans[m])  

C:\Users\kimru\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Create table 4

These next function prints out a row of the table.

In [7]:
def format_row(symbol, sym_name, data, models, fmt):
    t = r'\hspace{5mm}' + symbol + ' & '
    
    for c in models:
        fstr = '{0:' + fmt + '}'
        if data[c][sym_name] == '--':
            t = t + '--' + ' & '
        else:
            t = t + fstr.format(float(data[c][sym_name])) + ' & '

    t = t[:-2] + '\\\  \n'
    return t

In [8]:
fw = open('table_4.tex', 'w')

fw.write(r'\documentclass[12pt,leqno]{article}' + '\n')
fw.write(r'\usepackage{booktabs}' + '\n')
fw.write(r'\usepackage{siunitx}' + '\n')
fw.write(r'\usepackage[left=0.5in,right=0.5in,top=1in,bottom=1in]{geometry}' + '\n')

fw.write(r'\sisetup{input-decimal-markers = .,input-ignore = {,}, parse-numbers=false, group-separator={,}, group-four-digits = false}'+'\n')
fw.write(r'\begin{document}' + '\n')

fw.write(r'\begin{table}[tbp]' + '\n')
fw.write(r'\small \centering')
fw.write(r'\begin{tabular*}{0.8\textwidth}{l@{\extracolsep{\fill}}SSSS}' + '\n')
fw.write(r'\toprule & \multicolumn{3}{c}{Benchmark} & {No-cost} \\ \cline{2-4}' + '\n')
fw.write(r'Change & {Bond} & {Fin. Autarky} & {Complete} & {Bond} \\' + '\n')

fw.write(r'\multicolumn{1}{l}{Welfare} &  &  &  &  \\' + '\n')
fw.write(format_row('Home', 'welfareH', welfs, models, '0.2f'))
fw.write(format_row('Foreign', 'welfareF', welfs, models, '0.2f'))

fw.write(r'\multicolumn{1}{l}{SS Consumption} &  &  &  &  \\' + '\n')
fw.write(format_row('Home', 'C1_h', ss_changes, models, '0.2f'))
fw.write(format_row('Foreign', 'C1_f', ss_changes, models, '0.2f'))

fw.write(r'\multicolumn{1}{l}{SS Establishments} &  &  &  &  \\' + '\n')
fw.write(format_row('Home', 'NT1_h', ss_changes, models, '0.2f'))
fw.write(format_row('Foreign', 'NT1_f', ss_changes, models, '0.2f'))

fw.write(r'\bottomrule' + '\n')
fw.write(r'\end{tabular*}' + '\n')
fw.write(r'\end{table}'+'\n')

fw.write(r'\end{document}')
fw.close()

Done.